In [1]:
import numpy as np

import os
os.environ["CUDA_VISIBLE_DEVICES"]="6"

In [2]:
import sys
sys.path = list(set(sys.path + ['/home/work/notebook/']))

import imp

In [5]:
# loader
def load_idx2sid_sid2idx(fn):
    idx2sid = []
    sid2idx = {}
    with open(fn, 'r') as fd:
        for line in fd:
            elems = line.rstrip().split(',')
            if len(elems) != 3:
                continue
            idx2sid.append(elems[1])
            sid2idx[elems[1]] = int(elems[0])
    return idx2sid, sid2idx

idx2sid, sid2idx = load_idx2sid_sid2idx("../data/item_index.txt")
idx2sid[:10], list(sid2idx.items())[:10]

(['S3524288173452773650',
  'S7830293816090784321',
  'S12877487411019282198',
  'S3610462032989854826',
  'S4204279186363610708',
  'SIueZvem+memprCI=',
  'SIuS4pOWPquiAgeiZjiI=',
  'SIuaVsOm4reWtkCI=',
  'S7508112586544275801',
  'SIuaLlOiQneWNnCI='],
 [('S3524288173452773650', 0),
  ('S7830293816090784321', 1),
  ('S12877487411019282198', 2),
  ('S3610462032989854826', 3),
  ('S4204279186363610708', 4),
  ('SIueZvem+memprCI=', 5),
  ('SIuS4pOWPquiAgeiZjiI=', 6),
  ('SIuaVsOm4reWtkCI=', 7),
  ('S7508112586544275801', 8),
  ('SIuaLlOiQneWNnCI=', 9)])

In [6]:
### Load meta dict
import requests
ITEM_POOL_DIR = '../data/item_metadata/'
def load_item_title_dict(fn):
    import json
    """
        load sid title
    """
    sid2title = {}
    sid2url = {}
    with open(fn, 'r') as fd:
        for line in fd:
            line = line.rstrip()
            elems = line.split('\t')
            sid = elems[0]
            meta = json.loads(elems[1])
            homefeed_metadata = meta['candidateItem']['[rus.cell.PbHcCandidateItem.homecardData.homecardData]']
            sid2title[sid] = homefeed_metadata['title']
            if 'shortVideoUrl' in homefeed_metadata:
                sid2url[sid] = homefeed_metadata['shortVideoUrl']
    return sid2title, sid2url

sid2title, sid2url = load_item_title_dict(ITEM_POOL_DIR + 'item_meta_json.txt')

def get_sid_title_remote(sid, sid2title=sid2title, sid2url=sid2url):
    url_base = "http://gzhl-ns-orp088-hsc1701.gzhl.baidu.com:8080/p/item_meta_l/"
    resp = requests.get("{}{}".format(url_base, sid)).json()
    try:
        meta = resp['model']['data']['item_meta_l']
        homefeed_metadata = meta['candidateItem']['[rus.cell.PbHcCandidateItem.homecardData.homecardData]']
        sid2title[sid] = homefeed_metadata['title']
        if 'shortVideoUrl' in homefeed_metadata:
            sid2url[sid] = homefeed_metadata['shortVideoUrl']
        return sid2title[sid]
    except Exception as e:
        return ""

def get_sid_title(sid, sid2title=sid2title, sid2url=sid2url):
    if sid not in sid2title:
        try:
            return get_sid_title_remote(sid, sid2title=sid2title, sid2url=sid2url)
        except Exception as e:
            return ''
    return sid2title[sid]

def get_sid_url(sid, sid2url=sid2url):
    if sid not in sid2url:
        return ''
    return sid2url[sid]

In [7]:
import json
def get_sid_play_url(sid):
    base_url = "http://gzbh-bce-ais-duer-online-006.gzbh.baidu.com:8200/duer_video/video_track/_search?q=video_id:"
    es_url = "{}{}".format(base_url, sid[1:])
    play_url = ""
    try:
        play_url = requests.get(es_url).json()["hits"]["hits"][0]["_source"]["self_build_url_http"]
    except Exception as e:
        pass
    return play_url

In [8]:
get_sid_play_url("S3524288173452773650")

'http://vd3.bdstatic.com/mda-idic1g8j96x2mz4v/sc/mda-idic1g8j96x2mz4v.mp4?playlist=%5B%22hd%22%2C%22sc%22%5D'

In [9]:
get_sid_title("S3524288173452773650"), get_sid_url("S3524288173452773650"), get_sid_title_remote("S3524288173452773650", sid2title=sid2title, sid2url=sid2url)

('广场舞精选《小薇小薇你在何方》',
 'http://vd3.bdstatic.com/mda-idic41nrmjv8g44r/mda-idic41nrmjv8g44r.mp4',
 '广场舞精选《小薇小薇你在何方》')

In [10]:
all_data = np.load('../data/mockuid_sididx_ts.npy')
train_data = all_data[20000:]
test_data = all_data[:20000]
train_data[:10], test_data[:10]

(array([(311,  15641, 1550793231), (311,  29825, 1550793290),
        (311, 336378, 1550793503), (311,  21754, 1550793661),
        (311,  73108, 1550793824), (311,  36350, 1550793938),
        (312,   5597, 1549011684), (312,   1806, 1549015392),
        (312,   1554, 1549015491), (312,   6142, 1549016127)],
       dtype=[('user_id', '<i8'), ('item_id', '<i8'), ('ts', '<i8')]),
 array([(0,  4401, 1548911253), (0, 11897, 1548911258),
        (0, 31859, 1548911318), (0, 15850, 1548911333),
        (0, 18266, 1548911396), (0, 39389, 1548911511),
        (0,  1021, 1548938672), (0,   989, 1548939353),
        (0,   885, 1548939495), (0,   993, 1548939609)],
       dtype=[('user_id', '<i8'), ('item_id', '<i8'), ('ts', '<i8')]))

In [11]:
total_users = max(set(list(train_data['user_id']) + list(test_data['user_id']))) + 1
total_items = max(set(list(train_data['item_id']) + list(test_data['item_id']))) + 1
print(total_users, total_items)

202826 386871


In [12]:
# create dataset instances
import recsys.dataset as dataset
train_dataset = dataset.Dataset(train_data, total_users, total_items, sortby='ts', name='Train')
test_dataset = dataset.Dataset(test_data, total_users, total_items, sortby='ts', name='Test')

In [13]:
import numpy as np
import tensorflow as tf
import recsys.recommenders.recommender_base as recommender_base
import recsys.modules.extractions.embedding_layer as embedding_layer
import recsys.modules.interactions.mlp_softmax as mlp_softmax
import recsys.modules.fusions.variable_average as variable_average
import recsys.modules.fusions.concatenate as concatenate
def MlpRec(batch_size, dim_item_embed, max_seq_len, total_items,
        l2_reg_embed=None, l2_reg_mlp=None, dropout=None, init_model_dir=None,
        save_model_dir='MlpRecShortVideo', train=True, serve=False):
    
    rec = recommender_base.Recommender(init_model_dir=init_model_dir,
                      save_model_dir=save_model_dir, train=train, serve=serve)
    
    @rec.traingraph.inputgraph(outs=['seq_item_id', 'seq_len', 'label'])
    def train_input_graph(subgraph):
      
        subgraph['seq_item_id'] = tf.placeholder(tf.int32, 
                                      shape=[batch_size, max_seq_len],
                                      name='seq_item_id')
        subgraph['seq_len'] = tf.placeholder(tf.int32, 
                                      shape=[batch_size], 
                                      name='seq_len')
        subgraph['label'] = tf.placeholder(tf.int32, 
                                      shape=[batch_size], 
                                      name='label')
        
        subgraph.register_global_input_mapping({'seq_item_id': subgraph['seq_item_id'],
                                                'seq_len': subgraph['seq_len'],
                                                'label': subgraph['label']})
        
    @rec.servegraph.inputgraph(outs=['seq_item_id', 'seq_len'])
    def serve_input_graph(subgraph):
        subgraph['seq_item_id'] = tf.placeholder(tf.int32, 
                                      shape=[None, max_seq_len],
                                      name='seq_item_id')
        subgraph['seq_len'] = tf.placeholder(tf.int32, 
                                      shape=[None],
                                      name='seq_len')
        subgraph.register_global_input_mapping({'seq_item_id': subgraph['seq_item_id'],
                                                'seq_len': subgraph['seq_len']})
    
    @rec.traingraph.itemgraph(ins=['seq_item_id'], outs=['seq_vec'])
    @rec.servegraph.itemgraph(ins=['seq_item_id'], outs=['seq_vec'])
    def item_graph(subgraph):
        _, subgraph['seq_vec']= embedding_layer.apply(l2_reg=l2_reg_embed,
                                      init='normal',
                                      id_=subgraph['seq_item_id'],
                                      shape=[total_items,dim_item_embed],
                                      subgraph=subgraph,
                                      scope='item')

    @rec.traingraph.fusiongraph(ins=['seq_vec', 'seq_len'], outs=['fusion_vec'])
    @rec.servegraph.fusiongraph(ins=['seq_vec', 'seq_len'], outs=['fusion_vec'])
    def fusion_graph(subgraph):
        item_repr = variable_average.apply(sequence=subgraph['seq_vec'], seq_len=subgraph['seq_len'])
        fusion_vec = concatenate.apply([item_repr])
        subgraph['fusion_vec'] = fusion_vec

    @rec.traingraph.interactiongraph(ins=['fusion_vec', 'label'])
    def train_interaction_graph(subgraph):
        mlp_softmax.apply(
            in_tensor=subgraph['fusion_vec'],
            dims=[dim_item_embed, total_items],
            l2_reg=l2_reg_mlp,
            labels=subgraph['label'],
            dropout=dropout,
            train=True,
            subgraph=subgraph,
            scope='MLPSoftmax')

    @rec.servegraph.interactiongraph(ins=['fusion_vec'])
    def serve_interaction_graph(subgraph):
        mlp_softmax.apply(
            in_tensor=subgraph['fusion_vec'],
            dims=[dim_item_embed, total_items],
            l2_reg=l2_reg_mlp,
            train=False,
            subgraph=subgraph,
            scope='MLPSoftmax')

    @rec.traingraph.optimizergraph
    def optimizer_graph(subgraph):
        losses = tf.add_n(subgraph.get_global_losses())
        optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
        subgraph.register_global_operation(optimizer.minimize(losses))

    @rec.traingraph.connector
    @rec.servegraph.connector
    def connect(graph):
        graph.itemgraph['seq_item_id'] = graph.inputgraph['seq_item_id']
        graph.fusiongraph['seq_len'] = graph.inputgraph['seq_len']
        graph.fusiongraph['seq_vec'] = graph.itemgraph['seq_vec']
        graph.interactiongraph['fusion_vec'] = graph.fusiongraph['fusion_vec']

    @rec.traingraph.connector.extend
    def train_connect(graph):
        graph.interactiongraph['label'] = graph.inputgraph['label']

    return rec

In [14]:
# hyperparameters
dim_item_embed = 50     # dimension of item embedding
max_seq_len = 50        # the maxium length of user's listen history
total_iter = int(1e6)   # iterations for training 
batch_size = 128        # training batch size
eval_iter = int(2*1e5)  # iteration of evaluation
save_iter = eval_iter   # iteration of saving model   

In [15]:
# create data samplers
import recsys.samplers.temporal_sampler as temporal_sampler

train_sampler = temporal_sampler.create_training_sampler(batch_size=batch_size, max_seq_len=max_seq_len,
    dataset=train_dataset, num_process=5)
test_sampler = temporal_sampler.create_evaluation_sampler(dataset=test_dataset, max_seq_len=max_seq_len)

In [16]:
# create evaluators
import recsys.metrics.auc as auc
import recsys.metrics.ndcg as ndcg
import recsys.metrics.recall as recall
import recsys.metrics.precision as precision

auc_evaluator = auc.AUC()
recall_evaluator = recall.Recall(recall_at=[20, 50, 100])

In [17]:
# create model
model = MlpRec(batch_size=batch_size,
                   total_items=train_dataset.total_items(),
                   max_seq_len=max_seq_len,
                   dim_item_embed=dim_item_embed,
                   save_model_dir='../MlpRecShortvideo',
                   train=True, 
                   serve=True)

In [18]:
# trainer
import recsys.model_trainer as model_trainer
trainer = model_trainer.ModelTrainer(model=model)

In [19]:
import datetime
print(datetime.datetime.now())

2019-03-07 15:09:08.092239


In [20]:
# train
trainer.train(total_iter=total_iter, 
                    eval_iter=eval_iter,
                    save_iter=save_iter,
                    train_sampler=train_sampler,
                    eval_samplers=[test_sampler], 
                    evaluators=[auc_evaluator, recall_evaluator])

[Training starts, total_iter: 1000000, eval_iter: 200000, save_iter: 200000]
[iter 200000] Model saved.
[iter 200000] loss: 8.475143
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from ../MlpRecShortvideo/model.ckpt
..(dataset: Test) AUC 0.933126321834738
..(dataset: Test) Recall 0.1794871794871795 0.2564102564102564 0.32371794871794873
[iter 400000] Model saved.
[iter 400000] loss: 7.589377
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from ../MlpRecShortvideo/model.ckpt
..(dataset: Test) AUC 0.9199223650958084
..(dataset: Test) Recall 0.1987179487179487 0.2948717948717949 0.3525641025641026
[iter 600000] Model saved.
[iter 600000] loss: 7.346894
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from ../MlpRecShortvideo/model.ckpt
..(dataset: Test) AUC 0.9150844085305274
..(dataset: Test) Recall 0.18269230769230768 0.2692307692307692 0.3557692307692308
[iter 800000] Model saved.
[iter 800000] loss: 7.202637
..(dataset: Test) evalu

In [21]:
import datetime
print(datetime.datetime.now())

2019-03-07 23:50:32.318215


In [22]:
# serve
serve_sampler = temporal_sampler.create_evaluation_sampler(dataset=test_dataset, max_seq_len=max_seq_len)

In [23]:
lbl, input_data = serve_sampler.next_batch()
lbl, input_data

([373],
 array([([   359, 221384, 277327,   7615,  65222,   1185,   6049,  10858, 127624,  10070,  25418,  24061, 313711,   9728,  11931,  10439,  40873,  18864,  78435,  43572,   3768,   4653,   4009,   4100,   2609,   2083,   3387,   1819,    930,   3329,   3814,   4264,   2187,    468,     79,   3163,   1845,   3884,   3520,   3997,   4269,   2802,   3795,    534,   4309,   4118,    602,   3363,    528,   3103], 50)],
       dtype=[('seq_item_id', '<i4', (50,)), ('seq_len', '<i4')]))

In [24]:
output_dict = model.serve(batch_data=input_data)
print("outputs:", output_dict)
predict_proba = output_dict['outputs'][0].ravel()
ind_largest = np.argsort(predict_proba)[-20:][::-1]
print("indices:", ind_largest)
print("probs:", predict_proba[ind_largest])


outputs: {'losses': [], 'outputs': [array([[ -9.177443 ,  -4.922169 ,  -4.7542458, ..., -16.834045 ,
        -16.836594 , -16.703777 ]], dtype=float32)]}
indices: [ 19412   2871  55644   6554   5380  10144  10705    670  31282  13386
  18490  17465  28138   3768 134293    610  22777  18379   2759 255473]
probs: [-1.421916  -1.5408653 -1.5417953 -1.5544977 -1.586319  -1.613105
 -1.6254263 -1.6811426 -1.6840894 -1.7012589 -1.7019238 -1.713649
 -1.7174006 -1.7235339 -1.729166  -1.7396202 -1.7450464 -1.7458403
 -1.7515821 -1.7591066]


In [25]:
# input items
for i in input_data[0][0][:input_data[0][1]][:10]:
    print(idx2sid[i], get_sid_title(idx2sid[i]), get_sid_url(idx2sid[i]))

S13623819049506121020 我的世界：这只苦力怕好像发烧了？没救了，拿去炒了吧！ http://vd3.bdstatic.com/mda-ima01524qsdsvb0x/mda-ima01524qsdsvb0x.mp4
S1493762141713707017 玩家合成星球吞噬之铲！随手挖几下，我的地图只剩下矿石了！ 
S864698008028002284 生活大冒险06：蜘蛛爬上巨大豆蔓，由此引发一出惨剧！ http://vd3.bdstatic.com/mda-ja7dfnbmhqvtg6bv/mda-ja7dfnbmhqvtg6bv.mp4
S13237465854963024167 我的世界星源幸运方块：两个解说被这个幸运方块吓得三分钟就关服 http://vd3.bdstatic.com/mda-jaqqu59dn54i84iy/mda-jaqqu59dn54i84iy.mp4
S11007996436191890697 幸运方块生存，木鱼开出无限伤害的大宝剑，谁还敢与他为敌？ http://vd3.bdstatic.com/mda-imixynbfcqpiewz0/mda-imixynbfcqpiewz0.mp4
S14436283028383310529 我的世界海洋版57：挖到8块钻石矿，利用时运附魔变成14块！ http://vd3.bdstatic.com/mda-ja3rss9wm6zu1h2j/mda-ja3rss9wm6zu1h2j.mp4
S3336457945675644344 生活大冒险02：玩家种下一颗魔豆，长出一株巨大豆蔓！ 
S7916075981418457291 真人我的世界：卷毛的女友脑袋咋就多个兔子，卷毛直接拿去烤兔肉 http://vd3.bdstatic.com/mda-imgrcbqhb2x4r68w/mda-imgrcbqhb2x4r68w.mp4
S8108096182645184886 我的世界：爆爆逛街竟购买惩罚药水，狗剩喝完让他干啥就干啥 http://vd3.bdstatic.com/mda-jbeqnpxf8bkniw0b/mda-jbeqnpxf8bkniw0b.mp4
S13430180518838351755 我的世界：每一个方块都不要轻易打破，不然你会获得“惊喜”！ htt

In [26]:
# predict query
pred_ind = ind_largest.tolist()
for i in pred_ind[:10]:
    print(idx2sid[i], get_sid_title(idx2sid[i]), get_sid_url(idx2sid[i]))

S16063928646603726866 我的世界搞笑：怪物们挑战恐怖奶奶真人游戏，把苦力怕吓坏了 http://vd3.bdstatic.com/mda-imnj90sv9gsfjyiw/mda-imnj90sv9gsfjyiw.mp4
S17813197308405879044 我的世界第二季433：我在林地府邸的二楼，找到一个大宝箱 http://vd3.bdstatic.com/mda-jadtb83d5ph17gts/mda-jadtb83d5ph17gts.mp4
S10888000542223709482 我的世界海洋版27：村民繁殖所完工！十分钟的时间就繁殖大量村民 http://vd3.bdstatic.com/mda-imadqr2ejtr3i3ak/mda-imadqr2ejtr3i3ak.mp4
S791751261610201399 我的世界拔刀76：满地的拔刀剑，我到底经历了什么！ http://vd3.bdstatic.com/mda-jb0jsmghr0d4tn6n/mda-jb0jsmghr0d4tn6n.mp4
S15142376063710724607 我的世界：怪物学院，挑战植物大战僵尸，这样玩好嗨哦 http://vd3.bdstatic.com/mda-jafnmg74p7y7sinn/mda-jafnmg74p7y7sinn.mp4
S1735674843208570565 我的世界联机空岛生存237：大背头穿过传送门，发现一个空岛村庄 http://vd3.bdstatic.com/mda-im5sqp5uf9045k8j/mda-im5sqp5uf9045k8j.mp4
S934599867772716901 MC动画怪物学院《欢乐跳瓶》，男生女生施展跳瓶组合技能 http://vd3.bdstatic.com/mda-im7tixkq2h6qbypd/mda-im7tixkq2h6qbypd.mp4
S5342644274128005191 迷你世界：妮妮不讲诚信，说好不许再坑人的，结果我又掉陷阱里了 http://vd3.bdstatic.com/mda-ja1yrhepw190i21z/mda-ja1yrhepw190i21z.mp4
S8088276222657749191 我的世界钻石王国05：只要安装莲

In [30]:
# generate human evaluate excel
import pandas as pd
df = pd.DataFrame(columns=["mocked_uid", "flag", "title", "preview_url", "h5_url"])
def df_append(df, uid, flag, title, preview_url, h5_url):
    df.loc[len(df)] = uid, flag, title, preview_url, h5_url

In [31]:
def generate_excel(df):
    for mock_uid in range(100):
        print("Start uid: {}".format(mock_uid))
        lbl, input_data = serve_sampler.next_batch()
        if len(lbl) == 0: # retry
            lbl, input_data = serve_sampler.next_batch()
        output_dict = model.serve(batch_data=input_data)
        predict_proba = output_dict['outputs'][0].ravel()
        ind_largest = np.argsort(predict_proba)[-20:]

        for i in input_data[0][0][:input_data[0][1]][:10]:
            print("-- processing index: {}".format(i))
            title = get_sid_title(idx2sid[i])
            if len(title) == 0:
                continue
            df_append(df, mock_uid, "history", title, get_sid_url(idx2sid[i]), get_sid_play_url(idx2sid[i]))
        print("- finish input, start output")
        pred_ind = ind_largest.tolist()
        pred_ind.reverse()
        for i in pred_ind[:10]:
            title = get_sid_title(idx2sid[i])
            if len(title) == 0:
                continue
            df_append(df, mock_uid, "predict", title, get_sid_url(idx2sid[i]), get_sid_play_url(idx2sid[i]))
        print("Finish uid: {}".format(mock_uid))
        pass

In [32]:
generate_excel(df)
df.to_excel("sample_predict.xlsx")
print("DONE")

Start uid: 0
-- processing index: 20
-- processing index: 93331
-- processing index: 72560
-- processing index: 66927
-- processing index: 24569
-- processing index: 76180
-- processing index: 50225
-- processing index: 26078
-- processing index: 56205
-- processing index: 65653
- finish input, start output
Finish uid: 0
Start uid: 1
-- processing index: 158900
-- processing index: 112615
-- processing index: 109152
-- processing index: 172278
-- processing index: 255433
-- processing index: 363045
-- processing index: 118968
-- processing index: 28953
-- processing index: 41047
-- processing index: 97803
- finish input, start output
Finish uid: 1
Start uid: 2
-- processing index: 149737
-- processing index: 188073
-- processing index: 47120
-- processing index: 9
-- processing index: 140
-- processing index: 26
-- processing index: 46
-- processing index: 63
-- processing index: 90
-- processing index: 92
- finish input, start output
Finish uid: 2
Start uid: 3
-- processing index: 256

- finish input, start output
Finish uid: 25
Start uid: 26
-- processing index: 1236
-- processing index: 1384
-- processing index: 1463
-- processing index: 1511
-- processing index: 1617
-- processing index: 1699
-- processing index: 1246
-- processing index: 1202
-- processing index: 1396
-- processing index: 1487
- finish input, start output
Finish uid: 26
Start uid: 27
-- processing index: 5990
-- processing index: 6278
-- processing index: 1951
-- processing index: 2346
-- processing index: 1371
-- processing index: 16836
-- processing index: 25835
-- processing index: 60076
-- processing index: 61800
-- processing index: 112619
- finish input, start output
Finish uid: 27
Start uid: 28
-- processing index: 5641
-- processing index: 34763
-- processing index: 12857
-- processing index: 2481
-- processing index: 15683
-- processing index: 22197
-- processing index: 9912
-- processing index: 67382
-- processing index: 22879
-- processing index: 25369
- finish input, start output
Fini

-- processing index: 43573
-- processing index: 18938
-- processing index: 1753
-- processing index: 15661
-- processing index: 29531
-- processing index: 13902
- finish input, start output
Finish uid: 51
Start uid: 52
-- processing index: 127631
-- processing index: 19764
-- processing index: 18075
-- processing index: 59427
-- processing index: 59090
-- processing index: 33752
-- processing index: 110266
-- processing index: 123204
-- processing index: 141662
- finish input, start output
Finish uid: 52
Start uid: 53
-- processing index: 145563
-- processing index: 10192
-- processing index: 590
-- processing index: 74
-- processing index: 14637
-- processing index: 47121
-- processing index: 2184
-- processing index: 18138
-- processing index: 3771
-- processing index: 52747
- finish input, start output
Finish uid: 53
Start uid: 54
-- processing index: 1397
-- processing index: 110267
-- processing index: 139796
-- processing index: 116315
-- processing index: 216721
-- processing in

-- processing index: 5875
-- processing index: 8192
-- processing index: 9912
-- processing index: 8857
-- processing index: 18985
-- processing index: 19055
-- processing index: 3509
-- processing index: 7229
- finish input, start output
Finish uid: 77
Start uid: 78
-- processing index: 14290
-- processing index: 134232
-- processing index: 28138
-- processing index: 90098
-- processing index: 1413
-- processing index: 8444
-- processing index: 203495
-- processing index: 90092
-- processing index: 2242
-- processing index: 1253
- finish input, start output
Finish uid: 78
Start uid: 79
-- processing index: 1940
-- processing index: 16217
-- processing index: 12377
-- processing index: 18762
-- processing index: 172295
-- processing index: 175204
-- processing index: 199345
-- processing index: 269529
-- processing index: 2663
-- processing index: 336335
- finish input, start output
Finish uid: 79
Start uid: 80
-- processing index: 4121
-- processing index: 2599
-- processing index: 15

IllegalCharacterError: 

In [27]:
# export
model.export(top_k=100)

INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: b'../MlpRecShortvideo_exported_pb/saved_model.pb'


In [28]:
# export dict
!mkdir -p "../item_index_dict"
with open("../item_index_dict/dict.txt", "w") as fd:
    for sid in idx2sid:
        fd.write("{}\n".format(sid))

!cat "../item_index_dict/dict.txt" | wc -l > "../item_index_dict/number.txt"
!date +%Y%m%d > "../item_index_dict/version.txt"

In [29]:
# predict using exported pb model
model.predict_pb(feed_name_dict={
    "seq_item_id": input_data["seq_item_id"],
    "seq_len": input_data["seq_len"]
}, top_k=100)

INFO:tensorflow:Restoring parameters from b'../MlpRecShortvideo_exported_pb/variables/variables'


{'y_values': array([-1.421916 , -1.5408653, -1.5417953, -1.5544977, -1.586319 ,
        -1.613105 , -1.6254263, -1.6811426, -1.6840894, -1.7012589,
        -1.7019238, -1.713649 , -1.7174006, -1.7235339, -1.729166 ,
        -1.7396202, -1.7450464, -1.7458403, -1.7515821, -1.7591066,
        -1.7956893, -1.8168865, -1.8495243, -1.8526919, -1.900927 ,
        -1.9035907, -1.9118919, -1.9344943, -1.9414221, -1.9465878,
        -1.9792655, -1.991722 , -1.9920348, -2.003535 , -2.0106564,
        -2.0159073, -2.0260854, -2.0432372, -2.0855207, -2.0911388,
        -2.111876 , -2.1120858, -2.1181931, -2.12425  , -2.125333 ,
        -2.1327407, -2.15067  , -2.1526155, -2.156107 , -2.156618 ,
        -2.159533 , -2.1632068, -2.1682012, -2.2066998, -2.2079928,
        -2.2104466, -2.2160575, -2.2258515, -2.226134 , -2.230709 ,
        -2.236197 , -2.2486012, -2.2487993, -2.2522514, -2.257697 ,
        -2.2629802, -2.2772796, -2.2787986, -2.293652 , -2.2943087,
        -2.2956648, -2.2965217, -2.3